# 第4章 はじめてのベイズ推論実習

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp
#import jax.random as random

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}
with open("rcParams.json") as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))
import seaborn as sns
import arviz as az
%matplotlib inline

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 4.1 問題設定
> 常に確率が一定で、前回の結果が次回に一切影響しないくじ引きがあります。\
> ある人がこのくじ引きを5回引いたところ、結果は「当たり、はずれ、はずれ、当たり、はずれ」でした。\
> 1回のくじ引きに当たる確率を $p$ とするとき、この $p$ の値を求めなさい。

## 4.2 最尤推定
ベイズ推論ではないので省略する。

## 4.3 ベイズ推論(確率モデル定義)
ベイズ推論の大きな流れは下記の通り。

序数 | 項目 | 説明 |
---- | ---- | ---- |
1 | データ準備 | 通常の機械学習と同じ。 ```pandas```/```NumPy```などで実施する。 |
2 | 確率モデル定義 | 確率変数の関係を確率モデル化し、```PyMC```によりう記述する。 |
3 | サンプリング | サンプリングデータの生成。```PyMV```の```sample```関数により実施する。 |
4 | 結果分析 | 主に```ArviZ```ライブラリを用いて、サンプリング結果を統計的に分析する。 |

**Step 1. データ準備**\
くじ引きの結果は「試行の結果」であるため、書籍の $X$ ではなく目的変数の $Y$ を使う。\
複数回の試行結果を1次元の配列 $Y \equiv (y_{0}, \cdots, y_{4})^{T}$ にデータをまとめる。

In [ ]:
Y = jnp.array([1, 0, 0, 1, 0], dtype = int)

**Step 2. 確率モデル定義**\
確率モデルをプログラミングするために、数式を使って整理する。

1. 前回の結果が次回に一切影響しないくじ引きである。
    * DataFrameの順番を入れ替えても分析結果は変わらない、という意味である。
    * 独立同分布, be independent and identically distributed, i.i.d という。
1. 確率が一定のくじ引きである。
    * 各試行の結果 $y_{i} \ (i = 0, \cdots, 4)$ が確率 $p$ のベルヌーイ分布に従うと仮定する。
    * $y_{i} \sim Bern(y∣p)$
1. くじ引きの確率 $p$ に関する情報は無い
    * 確率 $p$ の事前分布は一様分布 $[0,1]$ と仮定する。

PyMCやNumPyroのプログラミングは、上記のまとめを後ろから記述する。

In [ ]:
def model_Bernoulli(N, Y = None):
    '''
        第4章のくじ引きの確率モデル
    '''
    # 3. 確率 $p$ の事前分布は一様分布 $[0,1]$ と仮定する。
    p = numpyro.sample("p", dist.Uniform(low = 0, high = 1))
    # 2. $y_{i} \sim Bern(y∣p)$
    # ベクトル化
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Bernoulli(probs = p), obs = Y)

In [ ]:
model_args = {
    "N": len(Y),
    "Y": Y
}
try_render_model(model_Bernoulli, render_name = "Lottery (Bernoulli distribution)", **model_args)

## 4,4 ベイズ推論(サンプリング)
**Step 3. サンプリング**

In [ ]:
model_args = {
    "N": len(Y),
    "Y": Y
}
mcmc = run_mcmc(model_Bernoulli, num_chains = 4, num_warmup = 1000, num_samples = 1000, thinning = 1, seed = 42, **model_args)

## 4.5 ベイズ推論(結果分析)
**Step 4. 結果分析**
### 4.5.1 plot_trace関数

In [ ]:
az.plot_trace(mcmc, compact = False)
plt.tight_layout()

### 4.5.2 plot_posterior 関数

In [ ]:
ax = az.plot_posterior(mcmc)
ax.set_xlim(0, 1)
plt.suptitle("ベイズ推論結果 初期版")
plt.tight_layout()
plt.show()

### 4.5.3 summary関数

In [ ]:
summary = az.summary(mcmc)
display(summary)

## 4.6 ベイズ推論(二項分布バージョン)
4.5節のくじ引きの例はデータを集計した取り扱いのほうが多い(5回中2回当たりだった)。\
前節と同様に数式、プログラミングの順番でデータを分析する。

1. 前回の結果が次回に一切影響しないくじ引きを複数回試行した結果を集約した。
    * 当たりを引いた回数は二項分布 $y \sim Bi(y|p, n)$ に従うと仮定する。
1. くじ引きの確率 $p$ に関する情報は無い
    * 確率 $p$ の事前分布は一様分布 $[0,1]$ と仮定する。

In [ ]:
def model_Binomial(N, y = None):
    '''
        第4章のくじ引きの二項分布モデル
    '''
    # 2. 確率 $p$ の事前分布は一様分布 $[0,1]$ と仮定する。
    p = numpyro.sample("p", dist.Uniform(low = 0, high = 1))
    # 1. 当たりを引いた回数は二項分布 $y \sim Bi(y|p, n)$ に従うと仮定する。
    numpyro.sample("y", dist.Binomial(total_count = N, probs = p), obs = y)

In [ ]:
y = len(Y[Y > 0])
model_args = {
    "N": len(Y),
    "y": y
}
try_render_model(model_Binomial, render_name = "Lottery (Binomial distribution)", **model_args)

In [ ]:
y = len(Y[Y > 0])
model_args = {
    "N": len(Y),
    "y": y
}
mcmc = run_mcmc(model_Binomial, num_chains = 4, num_warmup = 1000, num_samples = 1000, thinning = 1, seed = 42, **model_args)

In [ ]:
ax = az.plot_posterior(mcmc)
ax.set_xlim(0, 1)
plt.suptitle("ベイズ推論結果 二項分布版")
plt.tight_layout()
plt.show()

In [ ]:
summary = az.summary(mcmc)
display(summary)

## 4.7 ベイズ推論(試行回数を増やす)

In [ ]:
model_args = {
    "N": 50,
    "y": 20
}
mcmc = run_mcmc(model_Binomial, num_chains = 4, num_warmup = 1000, num_samples = 1000, thinning = 1, seed = 42, **model_args)

In [ ]:
ax = az.plot_posterior(mcmc)
ax.set_xlim(0, 1)
plt.suptitle("試行回数を増やす (n=50)")
plt.tight_layout()
plt.show()

In [ ]:
summary = az.summary(mcmc)
display(summary)

## 4.8 ベイズ推論(事前分布の変更)

In [ ]:
def model_Binomial_modified(N, y = None):
    '''
        前節の二項分布モデルに関して、当たりの確率の事前知識があった場合のモデル
    '''
    # 2. 確率 $p$ の事前分布である一様分布の値域を事前知識に基づいて修正する。
    p = numpyro.sample("p", dist.Uniform(low = 0.1, high = 0.9))
    # 1. 当たりを引いた回数は二項分布 $y \sim Bi(y|p, n)$ に従うと仮定する。
    numpyro.sample("y", dist.Binomial(total_count = N, probs = p), obs = y)

In [ ]:
# 4.6節と同じ試行回数で検証する。
y = len(Y[Y > 0])
model_args = {
    "N": len(Y),
    "y": y
}
mcmc = run_mcmc(model_Binomial_modified, num_chains = 4, num_warmup = 1000, num_samples = 1000, thinning = 1, seed = 42, **model_args)

In [ ]:
ax = az.plot_posterior(mcmc)
ax.set_xlim(0, 1)
plt.suptitle("事前分布変更版 [0.1, 0.9], (n=5)")
plt.tight_layout()
plt.show()

In [ ]:
summary = az.summary(mcmc)
display(summary)

## 4.9 ベータ分布で直接確率分布を求める
ベイズ推論ではないので省略する。